In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import random
import os
import pickle

from Model_class import CNN_model, CNN_Tree_Classifier, CNN_Bagging, CNN_Boosting
from Fine_Tune import Hyperparameter_Tuning as Tuning

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
path = 'D:\\庫存健診開發\\data\\Training\\processed\\'
CNN_param = {'Conv_layer': [2, 3],
                'Dense_layer': [1, 2],
                'Dense': [64, 128, 256],
                'dropout_ratio': [0, 0.1, 0.15, 0.2, 0.25, 0.3],
                'filter': [64, 100, 128, 256],
                'kernel': [3, 5],
                'padding': ['causal'],
                'learning_rate': [0.001, 0.002, 0.003],
                'batch': [64, 128, 256, 512],
                'epochs': [30, 40, 50, 60]}
model_param = {'n_estimator': [5, 10],
                    'subsample': [0.75, 0.8, 0.85, 0.9]}


Optimization = Tuning(df_path=path, model='CNN_Bagging', CNNparam=CNN_param, Modelparam=model_param, max_iter=15, cluster_num=0)

In [3]:
Optimization.load_optimal_model(9)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence

In [5]:
print(Optimization.Evaluation())

[0.6564950492776901, 0.6555465059455937, '              precision    recall  f1-score   support\n\n        down       0.63      0.70      0.67     15014\n          up       0.68      0.61      0.65     15681\n\n    accuracy                           0.66     30695\n   macro avg       0.66      0.66      0.66     30695\nweighted avg       0.66      0.66      0.66     30695\n']


In [20]:
stock = list(set([sublist[1] for sublist in Optimization.optimal._Y_test]))
prediction = Optimization.optimal.predict()

In [23]:
acc_list = []
for i, s in enumerate(tqdm(stock)):
    r, acc, auc = Optimization.optimal.Separate_Evaluation('test', stock=s, prediction=prediction)
    if acc < 0.6:
        acc_list.append([s, acc])

In [24]:
acc_list

[['2912', 0.5770833333333333],
 ['2207', 0.5708333333333333],
 ['1476', 0.5104166666666666],
 ['3008', 0.5145833333333333],
 ['2474', 0.4625],
 ['2454', 0.5604166666666667],
 ['2330', 0.5875]]

In [4]:
for i in range(15):
    Optimization = Tuning(df_path=path, model='CNN_boosting', CNNparam=CNN_param, Modelparam=model_param, max_iter=15, cluster_num=0)
    Optimization.load_optimal_model(i+1)
    print(Optimization.Overall_Evaluation())

KeyboardInterrupt: 

In [14]:
for i in range(15):
    Optimization = Tuning(df_path=path, model='CNN_bagging', CNNparam=CNN_param, Modelparam=model_param, max_iter=15, cluster_num=0)
    Optimization.load_optimal_model(i+1)
    print(Optimization.Overall_Evaluation())

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence


(0.13846153846153847, 0.13846153846153847)


C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence


(0.09230769230769231, 0.07692307692307693)


C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence

...

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence


(0.15384615384615385, 0.16923076923076924)


C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



(0.13846153846153847, 0.13846153846153847)


In [4]:
Optimization.load_optimal_model(2)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence

In [53]:
r, ac, au = Optimization.optimal.Separate_Evaluation(target='test', year=2018, prediction=pred)
print(r)

              precision    recall  f1-score   support

        down       0.52      0.48      0.50      8100
          up       0.48      0.53      0.50      7570

    accuracy                           0.50     15670
   macro avg       0.50      0.50      0.50     15670
weighted avg       0.50      0.50      0.50     15670



In [54]:
r, ac, au = Optimization.optimal.Separate_Evaluation(target='test', year=2019, prediction=pred)
print(r)

              precision    recall  f1-score   support

        down       0.44      0.47      0.45      4994
          up       0.56      0.53      0.54      6251

    accuracy                           0.50     11245
   macro avg       0.50      0.50      0.50     11245
weighted avg       0.51      0.50      0.50     11245



In [6]:
print(Optimization.Evaluation(target='test')[2])

              precision    recall  f1-score   support

        down       0.64      0.67      0.66     15014
          up       0.67      0.64      0.65     15681

    accuracy                           0.65     30695
   macro avg       0.65      0.65      0.65     30695
weighted avg       0.66      0.65      0.65     30695



In [47]:
import json
jf = []
with open('C:\\Users\\011553\\Downloads\\bq-results-20191101-083340-meua4tm7exz1.json' , 'r') as f:
    for line in f.readlines():
        dic = json.loads(line)
        jf.append(dic)

In [38]:
jf[-1'predicted_Y'][0]['tables']['score']

SyntaxError: invalid syntax (<ipython-input-38-dcca864d8b7f>, line 1)

In [49]:
pred = []
for item in jf:
    if item['predicted_Y'][0]['tables']['score'] > item['predicted_Y'][1]['tables']['score']:
        pred.append(0)
    else:
        pred.append(1)

In [50]:
len(pred)

30695

In [48]:
len(jf)

30695

In [52]:
pred = np.array(pred)

In [61]:
true = []
for item in jf:
    true.append(int(item['Y']))

In [62]:
true.count(1)

15681

In [63]:
true = np.array(true)

In [5]:
feature_list = ['close', 'high', 'low', 'open',
       'vol', 'VWAP', 
       'ma7', 'ma21', '26ema', '12ema', 'MACD', 'upper_band', 'lower_band',
       'close_skew', 'close_kurtosis', 'pvt', 'ATR', 'RSI_15', 'SO%k5', 'SO%d3', 'foreign_buy',
       'investment_buy', 'dealer_buy', 'foreign_buy_skew',
       'foreign_buy_kurtosis', 'investment_buy_skew',
       'investment_buy_kurtosis', 'dealer_buy_skew', 'dealer_buy_kurtosis']

In [6]:
num = [569,566,542,571,540,513,537,484,455,561,555,573,20,426,564,568,574,544,572,543]

In [10]:
f_list = []
for item in num:
    f = feature_list[(item+1)%29]
    day = (item+1) // 29
    f_list.append(f'{f}_day{day}')

In [11]:
f_list

['SO%d3_day19',
 'ATR_day19',
 'investment_buy_day18',
 'investment_buy_day19',
 'SO%d3_day18',
 'investment_buy_day17',
 'ATR_day18',
 'investment_buy_day16',
 'investment_buy_day15',
 'upper_band_day19',
 'VWAP_day19',
 'foreign_buy_skew_day19',
 'investment_buy_day0',
 'investment_buy_day14',
 'close_kurtosis_day19',
 'SO%k5_day19',
 'foreign_buy_kurtosis_day19',
 'foreign_buy_skew_day18',
 'dealer_buy_day19',
 'dealer_buy_day18']

In [12]:
[f'{feature}_day{i+1}' for i in range(20) for feature in feature_list]

['close_day1',
 'high_day1',
 'low_day1',
 'open_day1',
 'vol_day1',
 'VWAP_day1',
 'ma7_day1',
 'ma21_day1',
 '26ema_day1',
 '12ema_day1',
 'MACD_day1',
 'upper_band_day1',
 'lower_band_day1',
 'close_skew_day1',
 'close_kurtosis_day1',
 'pvt_day1',
 'ATR_day1',
 'RSI_15_day1',
 'SO%k5_day1',
 'SO%d3_day1',
 'foreign_buy_day1',
 'investment_buy_day1',
 'dealer_buy_day1',
 'foreign_buy_skew_day1',
 'foreign_buy_kurtosis_day1',
 'investment_buy_skew_day1',
 'investment_buy_kurtosis_day1',
 'dealer_buy_skew_day1',
 'dealer_buy_kurtosis_day1',
 'close_day2',
 'high_day2',
 'low_day2',
 'open_day2',
 'vol_day2',
 'VWAP_day2',
 'ma7_day2',
 'ma21_day2',
 '26ema_day2',
 '12ema_day2',
 'MACD_day2',
 'upper_band_day2',
 'lower_band_day2',
 'close_skew_day2',
 'close_kurtosis_day2',
 'pvt_day2',
 'ATR_day2',
 'RSI_15_day2',
 'SO%k5_day2',
 'SO%d3_day2',
 'foreign_buy_day2',
 'investment_buy_day2',
 'dealer_buy_day2',
 'foreign_buy_skew_day2',
 'foreign_buy_kurtosis_day2',
 'investment_buy_skew_